### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load 
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
#School Summaries
school_count = len(school_data)
student_count = len(student_data)
district_budget = school_data["budget"].sum()
district_mathScore_avg = student_data["math_score"].mean()
district_readingScore_avg = student_data["reading_score"].mean()
district_percent_mathPassed = round((len(student_data[student_data["math_score"] > 69.9])/student_count)*100)
district_percent_readingPassed = round((len(student_data[student_data["reading_score"] > 69.9])/student_count)*100)
district_percent_passing = round((len(student_data[(student_data['math_score'] > 69.9) & (student_data['reading_score'] > 69.9)])/student_count)*100)

d = {'Number of Schools': [school_count],
     "Number of Students": [student_count], 
     "Avg Math Score":[district_mathScore_avg],
     "Avg Reading Score":[district_readingScore_avg],
     "% Passing Math": [district_percent_mathPassed],
     "% Passing Reading": [district_percent_readingPassed],
     "Overall Passing" : [district_percent_passing]
    }

district_summary = pd.DataFrame(data = d)
district_summary["Avg Math Score"] = district_summary["Avg Math Score"].map("{:.2f}".format)
district_summary["Avg Reading Score"] = district_summary["Avg Reading Score"].map("{:.2f}".format)
district_summary                                                  

,Number of Schools,Number of Students,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,Overall Passing
0,15,39170,78.99,81.88,75,86,65


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [3]:
school_data_complete.head()
school_groups = school_data_complete.groupby("School ID")
school_groups

In [4]:
#Dataframe for holding all school data

school_row_df = pd.DataFrame(columns= [
                               "Type",
                               "Total Students",
                               "Budget",
                               "Budget/Student",
                               "Math Avg",
                               "Reading Avg",
                               "Percent Passing Math",
                               "Percent Passing Reading",
                               "Overall Percent Passing" ])

schoolNames = []
schoolTypes = []
schoolsTotalStudents = []
schoolBudgets = []
schoolBudgetPerStudent = []
schoolsMathScoreAvg = []
schoolsReadingScoreAvg = []
schoolsPercentMathPassed = []
schoolsPercentReadingPassed = []
schoolsPercentPassing = []

#loop thru each group to collect information
i = 0
for name, group in school_groups:
    schoolName = group['school_name'].unique()
    schoolNames.append(schoolName[0])

    schoolType = group['type'].unique()
    schoolTypes.append(schoolType[0])
    
    schoolsTotalStudents.append(len(group))
    
    schoolBudget = group['budget'].unique()
    schoolBudgets.append(schoolBudget[0])
    
    schoolBudgetPerStudent.append(schoolBudget[0] / len(group))
    
    school_mathScore_avg = group["math_score"].mean()
    school_readingScore_avg = group["reading_score"].mean()
    school_percent_mathPassed = round((len(group[group["math_score"] > 69.9])/len(group))*100)
    school_percent_readingPassed = round((len(group[group["reading_score"] > 69.9])/len(group))*100)
    school_percent_passing = round((len(group[(group['math_score'] > 69.9) & (group['reading_score'] > 69.9)])/len(group))*100)
    schoolsMathScoreAvg.append(school_mathScore_avg)
    schoolsReadingScoreAvg.append(school_readingScore_avg)
    schoolsPercentMathPassed.append(school_percent_mathPassed)
    schoolsPercentReadingPassed.append(school_percent_readingPassed)
    schoolsPercentPassing.append(school_percent_passing)
    
    school_row = pd.Series({
                               "Type": schoolType[0],
                               "Total Students" :len(group),
                               "Budget": schoolBudget[0],
                               "Budget/Student":schoolBudget[0] / len(group),
                               "Math Avg":school_mathScore_avg,
                               "Reading Avg":school_readingScore_avg,
                               "Percent Passing Math":school_percent_mathPassed ,
                               "Percent Passing Reading": school_percent_readingPassed,
                               "Overall Percent Passing":school_percent_passing },name = schoolName[0])

    i +=1
   
    school_row_df = school_row_df.append(school_row)
    
    
#create a single dataframe with all of these statistics
schools_data_df = pd.DataFrame({"Name" :schoolNames,
                               "Type": schoolTypes,
                               "Total Students": schoolsTotalStudents,
                               "Budget": schoolBudgets,
                               "Budget/Student": schoolBudgetPerStudent,
                               "Math Avg":schoolsMathScoreAvg,
                               "Reading Avg": schoolsReadingScoreAvg,
                               "% Passing Math": schoolsPercentMathPassed,
                               "% Passing Reading": schoolsPercentReadingPassed,
                               "Overall % Passing": schoolsPercentPassing})

schools_data_df["Budget"] = schools_data_df["Budget"].map("${:.2f}".format)
schools_data_df["Budget/Student"] = schools_data_df["Budget/Student"].map("${:.2f}".format)
schools_data_df["Math Avg"] = schools_data_df["Math Avg"].map("{:.2f}".format)
schools_data_df["Reading Avg"] = schools_data_df["Reading Avg"].map("{:.2f}".format)
headerList = schools_data_df.columns.tolist()
schools_data_df.to_csv(r'analysis\schoolSummary.csv', header=headerList, index=None, sep=',', mode='a')
schools_data_df

,Name,Type,Total Students,Budget,Budget/Student,Math Avg,Reading Avg,% Passing Math,% Passing Reading,Overall % Passing
0,Huang High School,District,2917,$1910635.00,$655.00,76.63,81.18,66,81,54
1,Figueroa High School,District,2949,$1884411.00,$639.00,76.71,81.16,66,81,53
2,Shelton High School,Charter,1761,$1056600.00,$600.00,83.36,83.73,94,96,90
3,Hernandez High School,District,4635,$3022020.00,$652.00,77.29,80.93,67,81,54
4,Griffin High School,Charter,1468,$917500.00,$625.00,83.35,83.82,93,97,91
5,Wilson High School,Charter,2283,$1319574.00,$578.00,83.27,83.99,94,97,91
6,Cabrera High School,Charter,1858,$1081356.00,$582.00,83.06,83.98,94,97,91
7,Bailey High School,District,4976,$3124928.00,$628.00,77.05,81.03,67,82,55
8,Holden High School,Charter,427,$248087.00,$581.00,83.80,83.81,93,96,89
9,Pena High School,Charter,962,$585858.00,$609.00,83.84,84.04,95,96,91


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [5]:
top_performing_df = school_row_df.sort_values("Overall Percent Passing", ascending = False)
top_performing_df["Budget"] = top_performing_df["Budget"].map("${:.2f}".format)
top_performing_df["Budget/Student"] = top_performing_df["Budget/Student"].map("${:.2f}".format)
top_performing_df["Math Avg"] = top_performing_df["Math Avg"].map("{:.2f}".format)
top_performing_df["Reading Avg"] = top_performing_df["Reading Avg"].map("{:.2f}".format)
headerList = top_performing_df.columns.tolist()
top_performing_df.to_csv(r'analysis\SchoolPerformance.csv', header=headerList, index_header = "School", index=True, sep=',', mode='a')
top_performing_df.head()

TypeError: to_csv() got an unexpected keyword argument 'index_header'

## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [ ]:
bottom_performing_df = school_row_df.sort_values("Overall Percent Passing", ascending = True)
bottom_performing_df["Budget"] = bottom_performing_df["Budget"].map("${:.2f}".format)
bottom_performing_df["Budget/Student"] = bottom_performing_df["Budget/Student"].map("${:.2f}".format)
bottom_performing_df["Math Avg"] = bottom_performing_df["Math Avg"].map("{:.2f}".format)
bottom_performing_df["Reading Avg"] = bottom_performing_df["Reading Avg"].map("{:.2f}".format)
bottom_performing_df.head()

## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [ ]:

    
#create separate dfs for each grade
grade9 = school_data_complete.loc[school_data_complete["grade"] == "9th", :]
grade10 = school_data_complete.loc[school_data_complete["grade"] == "10th", :]
grade11 = school_data_complete.loc[school_data_complete["grade"] == "11th", :]
grade12 = school_data_complete.loc[school_data_complete["grade"] == "12th", :]
schoolGrade_row_df = pd.DataFrame()

#Process each grade--

#Grade 9
schoolGrade9_row_df = pd.DataFrame(columns = ["Math Avg Grade 9"])
i = 0
#groupby school
school_group9 = grade9.groupby("School ID")
for name, group in school_group9:
    #school name
    schoolName = group['school_name'].unique()
    mathScoreAvg = group['math_score'].mean()
    grade9_math = pd.Series({"school_name":schoolName[0], "Math Avg Grade 9" : mathScoreAvg},name = i)
    schoolGrade9_row_df = schoolGrade9_row_df.append(grade9_math)
    i +=1

#Grade 10
schoolGrade10_row_df = pd.DataFrame(columns = ["Math Avg Grade 10"])
i = 0
#groupby school
school_group10 = grade10.groupby("School ID")
for name, group in school_group10:
    #school name
    schoolName = group['school_name'].unique()
    mathScoreAvg = group['math_score'].mean()
    grade10_math = pd.Series({"school_name":schoolName[0],"Math Avg Grade 10" : mathScoreAvg},name = i)
    schoolGrade10_row_df = schoolGrade10_row_df.append(grade10_math)
    i +=1
    
#Grade 11
schoolGrade11_row_df = pd.DataFrame(columns = ["Math Avg Grade 11"])
i=0
#groupby school
school_group11 = grade11.groupby("School ID")
for name, group in school_group11:
    #school name
    schoolName = group['school_name'].unique()
    mathScoreAvg = group['math_score'].mean()
    grade11_math = pd.Series({"school_name":schoolName[0],"Math Avg Grade 11" : mathScoreAvg},name = i)
    schoolGrade11_row_df = schoolGrade11_row_df.append(grade11_math)
    i += 1


#Grade 12
schoolGrade12_row_df = pd.DataFrame(columns = ["Math Avg Grade 12"])
i = 0
#groupby school
school_group12 = grade12.groupby("School ID")
for name, group in school_group12:
    #school name
    schoolName = group['school_name'].unique()
    mathScoreAvg = group['math_score'].mean()
    grade12_math = pd.Series({"school_name":schoolName[0],"Math Avg Grade 12" : mathScoreAvg},name = i)
    schoolGrade12_row_df = schoolGrade12_row_df.append(grade12_math)
    i +=1
#in each group add these averages to a series containing schoolName, and each grade average
#add each series to the final dataframe
all_grade_math_avgs = pd.DataFrame()
all_grade_math_avgs = schoolGrade9_row_df

all_grade_math_avgs = all_grade_math_avgs.merge(schoolGrade10_row_df, on = 'school_name')
all_grade_math_avgs = all_grade_math_avgs.merge(schoolGrade11_row_df, on = 'school_name')
all_grade_math_avgs = all_grade_math_avgs.merge(schoolGrade12_row_df, on = 'school_name')


all_grade_math_avgs.set_index('school_name', inplace = True)
all_grade_math_avgs["Math Avg Grade 9"] = all_grade_math_avgs["Math Avg Grade 9"].map("{:.2f}".format)
all_grade_math_avgs["Math Avg Grade 10"] = all_grade_math_avgs["Math Avg Grade 10"].map("{:.2f}".format)
all_grade_math_avgs["Math Avg Grade 11"] = all_grade_math_avgs["Math Avg Grade 11"].map("{:.2f}".format)
all_grade_math_avgs["Math Avg Grade 12"] = all_grade_math_avgs["Math Avg Grade 12"].map("{:.2f}".format)
headerList = all_grade_math_avgs.columns.tolist()
all_grade_math_avgs.to_csv(r'analysis\MathGradePerformance.csv', header=headerList, index_label = 'School', index=True, sep=',', mode='a')

all_grade_math_avgs

## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [ ]:

#Process each grade--

#Grade 9
schoolGrade9_row_df = pd.DataFrame(columns = ["Reading Avg Grade 9"])
i = 0
#groupby school
school_group9 = grade9.groupby("School ID")
for name, group in school_group9:
    #school name
    schoolName = group['school_name'].unique()
    readingScoreAvg = group['reading_score'].mean()
    grade9_reading = pd.Series({"school_name":schoolName[0], "Reading Avg Grade 9" : readingScoreAvg},name = i)
    schoolGrade9_row_df = schoolGrade9_row_df.append(grade9_reading)
    i +=1

#Grade 10
schoolGrade10_row_df = pd.DataFrame(columns = ["Reading Avg Grade 10"])
i = 0
#groupby school
school_group10 = grade10.groupby("School ID")
for name, group in school_group10:
    #school name
    schoolName = group['school_name'].unique()
    readingScoreAvg = group['reading_score'].mean()
    grade10_reading = pd.Series({"school_name":schoolName[0],"Reading Avg Grade 10" : readingScoreAvg},name = i)
    schoolGrade10_row_df = schoolGrade10_row_df.append(grade10_reading)
    i +=1
    
#Grade 11
schoolGrade11_row_df = pd.DataFrame(columns = ["Reading Avg Grade 11"])
i=0
#groupby school
school_group11 = grade11.groupby("School ID")
for name, group in school_group11:
    #school name
    schoolName = group['school_name'].unique()
    readingScoreAvg = group['reading_score'].mean()
    grade11_reading = pd.Series({"school_name":schoolName[0],"Reading Avg Grade 11" : readingScoreAvg},name = i)
    schoolGrade11_row_df = schoolGrade11_row_df.append(grade11_reading)
    i += 1


#Grade 12
schoolGrade12_row_df = pd.DataFrame(columns = ["Reading Avg Grade 12"])
i = 0
#groupby school
school_group12 = grade12.groupby("School ID")
for name, group in school_group12:
    #school name
    schoolName = group['school_name'].unique()
    readingScoreAvg = group['reading_score'].mean()
    grade12_reading = pd.Series({"school_name":schoolName[0],"Reading Avg Grade 12" : readingScoreAvg},name = i)
    schoolGrade12_row_df = schoolGrade12_row_df.append(grade12_reading)
    i +=1
#in each group add these averages to a series containing schoolName, and each grade average
#add each series to the final dataframe
all_grade_reading_avgs = pd.DataFrame()
all_grade_reading_avgs = schoolGrade9_row_df

all_grade_reading_avgs = all_grade_reading_avgs.merge(schoolGrade10_row_df, on = 'school_name')
all_grade_reading_avgs = all_grade_reading_avgs.merge(schoolGrade11_row_df, on = 'school_name')
all_grade_reading_avgs = all_grade_reading_avgs.merge(schoolGrade12_row_df, on = 'school_name')


all_grade_reading_avgs.set_index('school_name', inplace = True)
all_grade_reading_avgs["Reading Avg Grade 9"] = all_grade_reading_avgs["Reading Avg Grade 9"].map("{:.2f}".format)
all_grade_reading_avgs["Reading Avg Grade 10"] = all_grade_reading_avgs["Reading Avg Grade 10"].map("{:.2f}".format)
all_grade_reading_avgs["Reading Avg Grade 11"] = all_grade_reading_avgs["Reading Avg Grade 11"].map("{:.2f}".format)
all_grade_reading_avgs["Reading Avg Grade 12"] = all_grade_reading_avgs["Reading Avg Grade 12"].map("{:.2f}".format)
headerList = all_grade_reading_avgs.columns.tolist()
all_grade_reading_avgs.to_csv(r'analysis\ReadingGradePerformance.csv', header=headerList,index_label = 'School', index=True, sep=',', mode='a')
all_grade_reading_avgs

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [ ]:
school_row_df.head(1)

In [ ]:
#time to bin -use school_row_df, created earlier

#bin by spending / student; create 4 bins
school_row_df.describe()


bins = [500, 600, 625, 650, 700]
group_names = ["<$600/Student", "$600-625/Student", "$625-650/Student",  "$650-700/Student"]

school_row_df['spending level'] = pd.cut(school_row_df['Budget/Student'], bins, labels=group_names)

school_row_df['spending level'].value_counts()
# now group by these levels
SpendingLevel_stats_df = pd.DataFrame(columns = ['Spending Level',"Math Avg","Reading Avg"])

school_spending_groups = school_row_df.groupby("spending level")
i=0
for name, group in school_spending_groups: 
    spendingLevel = group['spending level'].unique()
    spendingLevel = spendingLevel[0]
    readingScoreAvg = group['Reading Avg'].mean()
    mathScoreAvg = group['Math Avg'].mean()
    spendingLevel_stats = pd.Series({"Spending Level": spendingLevel, "Math Avg": mathScoreAvg, "Reading Avg" : readingScoreAvg},name = i)
    SpendingLevel_stats_df = SpendingLevel_stats_df.append(spendingLevel_stats)
    i +=1
    
SpendingLevel_stats_df["Reading Avg"] = SpendingLevel_stats_df["Reading Avg"].map("{:.2f}".format)
SpendingLevel_stats_df["Math Avg"] = SpendingLevel_stats_df["Math Avg"].map("{:.2f}".format)
print(SpendingLevel_stats_df)


## Scores by School Size

* Perform the same operations as above, based on school size.

In [ ]:
#bin by school size ; create 4 bins
#time to bin -use school_row_df, created earlier

#bin by Total Students; create 4 bins

print(school_row_df['Total Students'].min())
print(school_row_df['Total Students'].max())

bins = [0,1000, 2000, 3000, 5000]
group_names = ["<1000 Students", "1K - 2K Students", "2K - 3K Students",  "3K - 5K Students"]

school_row_df['School Size'] = pd.cut(school_row_df['Total Students'], bins, labels=group_names)

school_row_df['School Size'].value_counts()
# now group by these levels
SchoolSize_stats_df = pd.DataFrame(columns = ['School Size',"Math Avg","Reading Avg"])

school_size_groups = school_row_df.groupby("School Size")
i=0
for name, group in school_size_groups: 
    schoolSize = group['School Size'].unique()
    schoolSize = schoolSize[0]
    readingScoreAvg = group['Reading Avg'].mean()
    mathScoreAvg = group['Math Avg'].mean()
    schoolSize_stats = pd.Series({"School Size": schoolSize, "Math Avg": mathScoreAvg, "Reading Avg" : readingScoreAvg},name = i)
    SchoolSize_stats_df = SchoolSize_stats_df.append(schoolSize_stats)
    i +=1
    
SchoolSize_stats_df["Reading Avg"] = SchoolSize_stats_df["Reading Avg"].map("{:.2f}".format)
SchoolSize_stats_df["Math Avg"] = SchoolSize_stats_df["Math Avg"].map("{:.2f}".format)

print(SchoolSize_stats_df)


## Scores by School Type

In [ ]:
school_row_df.head()
#add a new column to map 
def label_type (row):
   if row['Type'] == 'District' :
      return 1
   if row['Type'] == 'Charter' :
      return 2
   return 99

school_row_df.apply (lambda row: label_type(row), axis=1)
school_row_df['type_label'] = school_row_df.apply (lambda row: label_type(row), axis=1)
school_row_df

* Perform the same operations as above, based on school type

In [ ]:
#bin by type of school; create 2 bins, only 2 types
#time to bin -use school_row_df, created earlier

#bin by type; create 2 bins
school_row_df['Type'].value_counts()
bins = [0,1,2]
group_names = ["District", "Charter"]

school_row_df['type_label'] = pd.cut(school_row_df['type_label'], bins, labels=group_names)

# now group by these levels
SchoolType_stats_df = pd.DataFrame(columns = ['School Type',"Math Avg","Reading Avg"])

school_type_groups = school_row_df.groupby("Type")
i=0
for name, group in school_type_groups: 
    schoolType = group['Type'].unique()
    schoolType = schoolType[0]
    readingScoreAvg = group['Reading Avg'].mean()
    mathScoreAvg = group['Math Avg'].mean()
    schoolType_stats = pd.Series({"School Type": schoolType, "Math Avg": round(mathScoreAvg), "Reading Avg" : round(readingScoreAvg)},name = i)
    SchoolType_stats_df = SchoolType_stats_df.append(schoolType_stats)
    i +=1
 
print(SchoolType_stats_df)
